In [6]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import h5py
import copy

import amici
from petab.C import *
import petab
import petab.C
import pypesto
import pypesto.petab
from pypesto.optimize import minimize
from pypesto.startpoint import uniform
from pypesto.engine import MultiProcessEngine, MultiThreadEngine
from pypesto.optimize.optimizer import FidesOptimizer
import pypesto.optimize as optimize

from pypesto.visualize import waterfall
from pypesto.visualize import parameters
from pypesto.visualize.model_fit import visualize_optimized_model_fit
from pypesto.visualize import profiles

import pypesto.profile as profile
from pypesto.optimize import ScipyOptimizer
from pypesto.profile.options import ProfileOptions

from pypesto.visualize.model_fit import visualize_optimized_model_fit, _get_simulation_rdatas

In [7]:
def hex_to_rgba_gradient(color1, color2, n):
    '''
    Create a gradient in rgba between two hex colors
    '''
    # Convert to rgba
    c1 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color1))
    c2 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color2))

    return [[(c1[i]*(n-j-1) + c2[i]*j)/(n-1) for i in range(4)] for j in range(n)]

# find the index for cut off based on Chi square distribution CI 95%
def find_cut_off_index(result, ci = 0.95):
    '''
    Find the cut off index for the data based on the Chi square distribution
    '''

    # calculate the chi square distribution
    cut_off_value = scipy.stats.chi2.ppf(ci, 1)

    # find the index
    best_fval = result.optimize_result.list[0].fval

    for i in range(len(result.optimize_result.list)):
        if result.optimize_result.list[i].fval > best_fval + cut_off_value:
            break
    
    return i - 1

def find_cut_off_x_trace(result, ci = 0.95, flatten = True):

    cut_off_value = scipy.stats.chi2.ppf(ci, 1)
    best_fval = result.optimize_result.list[0].fval

    # store the optimized x trace that are below the cut off value
    x_trace_within_cut_off = []
    if flatten:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off.append(x_trace[j])
    else:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            x_trace_within_cut_off_i = []
            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off_i.append(x_trace[j])
            x_trace_within_cut_off.append(x_trace_within_cut_off_i)

    return x_trace_within_cut_off

In [8]:
# Plot setting
plt.rcParams['font.size'] = 30

dpi = 100
wid = int(2560/dpi)
hei = int(1600/dpi)

In [9]:
# number of optimization runs
n_runs = 5000

In [ ]:
# Define the folder where you want to save the figures
folder_path = "/Users/yuhongliu/Documents/OV/figures/ifac_individual_hidden_model/n"+str(n_runs)+"/"

# If the folder does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# optimization
hierarchical = True

petab_yaml = 'petab_files/individual_start.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

np.random.seed(500)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=hierarchical,
        model_name=f"INDIVIDUAL_START_Model",
    ).create_problem(force_compile=True)

problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")

In [ ]:
loading = True

if loading:
    # load result history from file
    result = pypesto.store.read_result('optimization_history/n'+ str(n_runs) +'.hdf5')

else:
    # optimize the model
    result = minimize(
        problem=problem,
        optimizer=FidesOptimizer(verbose=False, options={'maxiter': 5000}),
        n_starts=n_runs,
        engine=MultiProcessEngine(),
        # startpoint_method=uniform,
        history_options = pypesto.HistoryOptions(trace_record=True, storage_file='optimization_history/n'+ str(n_runs) +'.hdf5'),
        filename='optimization_history/n'+ str(n_runs) +'.hdf5',
    )

# print result summary
print(result.summary())

In [12]:
parameters_from_result = dict(zip(problem.x_names, result.optimize_result.list[0]['x']))

In [ ]:
# Scale all parameters and put them into a dictionary
scaled_parameters = {key: 10**value for key, value in parameters_from_result.items()}

# Print the scaled parameters
print("Scaled parameters:")
for key, value in scaled_parameters.items():
    print(f"{key}: {value}")

# Obtain data and visualize the fitting result

In [14]:
return_dict = problem.objective(result.optimize_result.list[0].x, return_dict=True)
rdatas = return_dict['rdatas']
edatas = problem.objective.edatas
x_axis = [edata.id for edata in edatas]
simulation = [rdata.y.reshape(5, -1)[:,0] for rdata in rdatas]
data = [np.array(edata.getObservedData()) for edata in edatas]

In [ ]:
# visualize the fitting result by comparing the simulation and the data
# separate into vvDD and ctrl with two subplots

# plot the ctrl data and simulation
plt.figure(figsize=(10, 8))
for i in range(10):
    plt.plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='red', alpha=0.3, label='data' if i == 0 else "")
    plt.plot(np.array([3,4,5,6,7]), simulation[i], 
         marker='*', linestyle='-', color='red', alpha=1, label='simulation' if i == 0 else "")
plt.xlabel('Time (days)')
plt.ylabel('Tumor Volume ($\mu m^3$)')
plt.title('Tumor Volume vs Time for PBS Conditions')
plt.legend()
plt.grid(False)
plt.xticks([3, 4, 5, 6, 7])
plt.tight_layout()
plt.savefig(folder_path + 'ctrl_fit.pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# visualize the fitting result by comparing the simulation and the data
# separate into vvDD and ctrl with two subplots

# plot the ctrl data and simulation
plt.figure(figsize=(10, 8))
for i in range(10):
    plt.plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='red', alpha=0.3, label='data' if i == 0 else "")
    plt.plot(np.array([3,4,5,6,7]), simulation[i], 
         marker='*', linestyle='-', color='red', alpha=1, label='simulation' if i == 0 else "")
    plt.xlabel('Time (days)')
    plt.ylabel('Tumor Volume ($\mu m^3$)')
    plt.title('Tumor Volume vs Time for PBS Conditions')
    plt.legend()
    plt.grid(False)
    plt.xticks([3, 4, 5, 6, 7])
    plt.tight_layout()
    # plt.savefig(folder_path + 'ctrl_fit.pdf', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# plot the vvDD data and simulation
plt.figure(figsize=(10, 8))
for i in range(10, 20):
    plt.plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='blue', alpha=0.3, label='data' if i == 10 else "")
    plt.plot(np.array([3,4,5,6,7]), simulation[i], 
         marker='*', linestyle='-', color='blue', alpha=1, label='simulation' if i == 10 else "")
plt.xlabel('Time (days)')
plt.ylabel('Tumor Volume ($\mu m^3$)')
plt.title('Tumor Volume vs Time for vvDD Conditions')
plt.legend()
plt.grid(False)
plt.xticks([3, 4, 5, 6, 7])
plt.tight_layout()
plt.savefig(folder_path + 'vvDD_fit.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# plot the vvDD data and simulation
plt.figure(figsize=(10, 8))
for i in range(10, 20):
    plt.plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='blue', alpha=0.3, label='data' if i == 10 else "")
    plt.plot(np.array([3,4,5,6,7]), simulation[i], 
         marker='*', linestyle='-', color='blue', alpha=1, label='simulation' if i == 10 else "")
    plt.xlabel('Time (days)')
    plt.ylabel('Tumor Volume ($\mu m^3$)')
    plt.title('Tumor Volume vs Time for vvDD Conditions')
    plt.legend()
    plt.grid(False)
    plt.xticks([3, 4, 5, 6, 7])
    plt.tight_layout()
    # plt.savefig(folder_path + 'vvDD_fit.pdf', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# plt.figure(figsize=(14, 8))

for i in range(10):
    plt.plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='red', alpha=0.3, label='data - ctrl' if i == 0 else "")
    plt.plot(np.array([3,4,5,6,7]), simulation[i], 
         marker='*', linestyle='-', color='red', alpha=1, label='simulation - ctrl' if i == 0 else "")

for i in range(10, 20):
    plt.plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='blue', alpha=0.3, label='data - vvDD' if i == 0 else "")
    plt.plot(np.array([3,4,5,6,7]), simulation[i], 
         marker='*', linestyle='-', color='blue', alpha=1, label='simulation - vvDD' if i == 0 else "")

plt.xlabel('Time (days)')
plt.ylabel('Tumor Volume ($\mu m^3$)')
plt.title('Tumor Volume vs Time for ctrl and vvDD Conditions')
plt.legend()
plt.grid(False)
plt.xticks([3, 4, 5, 6, 7])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  
plt.tight_layout()
plt.savefig(folder_path + 'ctrl_vvDD_fit.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# calculate the mean and std of the data within ctrl and vvDD groups and compare again with the simulation

# Calculate mean and standard error for PBS
pbs_mean = np.array(data[0:10]).mean(axis=0)
pbs_se = np.array(data[0:10]).std(axis=0)
# Calculate mean and standard error for vvDD
vvDD_mean = np.array(data[10:20]).mean(axis=0)
vvDD_se = np.array(data[10:20]).std(axis=0)
# Plot both provided and calculated data
plt.figure(figsize=(12, 8))
# Plot provided data
plt.errorbar(np.array([3,4,5,6,7]), pbs_mean, 
             yerr=[pbs_mean - pbs_se],
             fmt='o--', color='red', alpha=0.5, ecolor='lightcoral', capsize=5, label='data - ctrl')
plt.errorbar(np.array([3,4,5,6,7]), vvDD_mean, 
             yerr=[vvDD_mean - vvDD_se],
             fmt='o--', color='blue', alpha=0.5, ecolor='lightblue', capsize=5, label='data - vvDD')

# Calculate mean and standard error for PBS
pbs_mean_sim = np.array(simulation[0:10]).mean(axis=0)
pbs_se_sim = np.array(simulation[0:10]).std(axis=0)
# Calculate mean and standard error for vvDD
vvDD_mean_sim = np.array(simulation[10:20]).mean(axis=0)
vvDD_se_sim = np.array(simulation[10:20]).std(axis=0)
# Plot simulated data
plt.errorbar(np.array([3,4,5,6,7]), pbs_mean_sim, 
             yerr=[pbs_mean_sim - pbs_se_sim],
             fmt='*-', color='red', alpha=1, ecolor='red', capsize=5, label='simulation - ctrl')
plt.errorbar(np.array([3,4,5,6,7]), vvDD_mean_sim, 
             yerr=[vvDD_mean_sim - vvDD_se_sim],
             fmt='*-', color='blue', alpha=1, ecolor='blue', capsize=5, label='simulation - vvDD')

plt.xlabel('Time (days)')
plt.ylabel('Tumor Volume ($\mu$ $m^3$)') 
plt.title('Mean Tumor Volume with Standard Deviation vs Time')
plt.legend()
plt.grid(False)
plt.xticks([3, 4, 5, 6, 7])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  
plt.tight_layout()
plt.savefig(folder_path + 'mean_tumor_volume_with_std.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data
file_path = '/Users/yuhongliu/Documents/OV/me_models/first_version/monolix_models/monolix_v1/DataFile/indfits_data.csv'
monolix_data = pd.read_csv(file_path)

# Convert columns to numeric
monolix_data['TIME'] = pd.to_numeric(monolix_data['TIME'], errors='coerce')
monolix_data['indivPred_mode'] = pd.to_numeric(monolix_data['indivPred_mode'], errors='coerce')

# Plotting
plt.figure(figsize=(10, 6))

# Iterate through each unique ID
for id in monolix_data['ID'].unique():
    individual_monolix_data = monolix_data[monolix_data['ID'] == id]
    color = 'blue' if individual_monolix_data['GROUP'].iloc[0] == 'vvDD' else 'red'
    
    # Plot the monolix_data
    plt.plot(individual_monolix_data['TIME'], individual_monolix_data['indivPred_mode'], alpha=0.2, color=color)

# Calculate the mean of indivPred_mode at each time point for ID 1-10 and ID 11-20
mean_id_1_10 = monolix_data[monolix_data['ID'].between(1, 10)].groupby('TIME')['indivPred_mode'].mean()
mean_id_11_20 = monolix_data[monolix_data['ID'].between(11, 20)].groupby('TIME')['indivPred_mode'].mean()

# Plot the means
# Calculate the standard deviation of indivPred_mode at each time point for ID 1-10 and ID 11-20
std_id_1_10 = monolix_data[monolix_data['ID'].between(1, 10)].groupby('TIME')['indivPred_mode'].std()
std_id_11_20 = monolix_data[monolix_data['ID'].between(11, 20)].groupby('TIME')['indivPred_mode'].std()

# Filter the means and standard deviations for the specified time points
time_points = [0, 1, 2, 3, 4]
mean_id_1_10_filtered = mean_id_1_10[mean_id_1_10.index.isin(time_points)]
mean_id_11_20_filtered = mean_id_11_20[mean_id_11_20.index.isin(time_points)]
std_id_1_10_filtered = std_id_1_10[std_id_1_10.index.isin(time_points)]
std_id_11_20_filtered = std_id_11_20[std_id_11_20.index.isin(time_points)]

# Plot the means with standard deviation as error bars
plt.errorbar(mean_id_1_10_filtered.index, mean_id_1_10_filtered.values, yerr=std_id_1_10_filtered.values, label='Mean vvDD', color='blue', linewidth=2, capsize=5)
plt.errorbar(mean_id_11_20_filtered.index, mean_id_11_20_filtered.values, yerr=std_id_11_20_filtered.values, label='Mean ctrl', color='red', linewidth=2, capsize=5)

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('indivPred_mode')
plt.title('Individual Predictions by ID')
plt.legend()
plt.show()

In [ ]:
# Calculate mean and standard error for PBS
pbs_mean = np.array(data[0:10]).mean(axis=0)
pbs_se = np.array(data[0:10]).std(axis=0)
# Calculate mean and standard error for vvDD
vvDD_mean = np.array(data[10:20]).mean(axis=0)
vvDD_se = np.array(data[10:20]).std(axis=0)

# Plot both provided and calculated data
plt.figure(figsize=(20, 8))
# Plot provided data
plt.errorbar(np.array([0, 1,2, 3, 4]), pbs_mean, 
             yerr=[pbs_mean - pbs_se],
             fmt='o--', color='red', alpha=0.5, ecolor='lightcoral', capsize=5, label='data - ctrl')
plt.errorbar(np.array([0, 1,2, 3, 4]), vvDD_mean, 
             yerr=[vvDD_mean - vvDD_se],
             fmt='o--', color='blue', alpha=0.5, ecolor='lightblue', capsize=5, label='data - vvDD')

# Calculate mean and standard error for PBS
pbs_mean_sim = np.array(simulation[0:10]).mean(axis=0)
pbs_se_sim = np.array(simulation[0:10]).std(axis=0)
# Calculate mean and standard error for vvDD
vvDD_mean_sim = np.array(simulation[10:20]).mean(axis=0)
vvDD_se_sim = np.array(simulation[10:20]).std(axis=0)
# Plot simulated data
plt.errorbar(np.array([0,1,2,3,4]), pbs_mean_sim, 
             yerr=[pbs_mean_sim - pbs_se_sim],
             fmt='*-', color='red', alpha=1, ecolor='red', capsize=5, label='simulation - ctrl')
plt.errorbar(np.array([0,1,2,3,4]), vvDD_mean_sim, 
             yerr=[vvDD_mean_sim - vvDD_se_sim],
             fmt='*-', color='blue', alpha=1, ecolor='blue', capsize=5, label='simulation - vvDD')

plt.xlabel('Time (days)')
plt.ylabel(r'Tumor Volume ($\mu$ $m^3$)') 
plt.grid(False)
plt.xticks([0, 1, 2, 3, 4])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  

# Load the data
file_path = '/Users/yuhongliu/Documents/OV/me_models/first_version/monolix_models/monolix_v1/DataFile/indfits_data.csv'
monolix_data = pd.read_csv(file_path)

# Convert columns to numeric
monolix_data['TIME'] = pd.to_numeric(monolix_data['TIME'], errors='coerce')
monolix_data['indivPred_mode'] = pd.to_numeric(monolix_data['indivPred_mode'], errors='coerce')

# Iterate through each unique ID
# for id in monolix_data['ID'].unique():
#     individual_monolix_data = monolix_data[monolix_data['ID'] == id]
#     color = 'purple' if individual_monolix_data['GROUP'].iloc[0] == 'vvDD' else 'yellow'
    
#     # Plot the monolix_data
#     plt.plot(individual_monolix_data['TIME'], individual_monolix_data['indivPred_mode'], alpha=0.5, color=color)

# Calculate the mean of indivPred_mode at each time point for ID 1-10 and ID 11-20
mean_id_1_10 = monolix_data[monolix_data['ID'].between(1, 10)].groupby('TIME')['indivPred_mode'].mean()
mean_id_11_20 = monolix_data[monolix_data['ID'].between(11, 20)].groupby('TIME')['indivPred_mode'].mean()

# Plot the means
# Calculate the standard deviation of indivPred_mode at each time point for ID 1-10 and ID 11-20
std_id_1_10 = monolix_data[monolix_data['ID'].between(1, 10)].groupby('TIME')['indivPred_mode'].std()
std_id_11_20 = monolix_data[monolix_data['ID'].between(11, 20)].groupby('TIME')['indivPred_mode'].std()

# Filter the means and standard deviations for the specified time points
time_points = [0, 1, 2, 3, 4]
mean_id_1_10_filtered = mean_id_1_10[mean_id_1_10.index.isin(time_points)]
mean_id_11_20_filtered = mean_id_11_20[mean_id_11_20.index.isin(time_points)]
std_id_1_10_filtered = std_id_1_10[std_id_1_10.index.isin(time_points)]
std_id_11_20_filtered = std_id_11_20[std_id_11_20.index.isin(time_points)]

# Plot the means with standard deviation as error bars
plt.errorbar(mean_id_1_10_filtered.index, mean_id_1_10_filtered.values, yerr=std_id_1_10_filtered.values, label='ME vvDD', color='green', linewidth=2, capsize=5)
plt.errorbar(mean_id_11_20_filtered.index, mean_id_11_20_filtered.values, yerr=std_id_11_20_filtered.values, label='ME ctrl', color='orange', linewidth=2, capsize=5)

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('indivPred_mode')
plt.title('Individual Predictions by ID')
plt.legend(bbox_to_anchor=(1.25, 1), loc='upper left')
plt.tight_layout()
plt.savefig(folder_path + 'mean_tumor_volume_with_std.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create residual plots for PBS conditions
fig, axs = plt.subplots(2, 5, figsize=(25, 10))
fig.suptitle('Residuals for PBS Conditions', fontsize=30)

for i in range(10):

    residuals = data[i] - simulation[i]
    axs[i // 5, i % 5].plot(np.array([3, 4, 5, 6, 7]), residuals, marker='o', linestyle='-', color='red')
    axs[i // 5, i % 5].set_title(f'Individual {i+1}')
    axs[i // 5, i % 5].set_xlabel('Time (days)')
    axs[i // 5, i % 5].set_ylabel('Residuals')
    axs[i // 5, i % 5].grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(folder_path + 'residuals_pbs_conditions.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Create residual plots for vvDD conditions
fig, axs = plt.subplots(2, 5, figsize=(25, 10))
fig.suptitle('Residuals for vvDD Conditions', fontsize=30)

for i in range(10, 20):
    residuals = data[i] - simulation[i]
    axs[(i-10) // 5, (i-10) % 5].plot(np.array([3, 4, 5, 6, 7]), residuals, marker='o', linestyle='-', color='blue')
    axs[(i-10) // 5, (i-10) % 5].set_title(f'Individual {i-9}')
    axs[(i-10) // 5, (i-10) % 5].set_xlabel('Time (days)')
    axs[(i-10) // 5, (i-10) % 5].set_ylabel('Residuals')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(folder_path + 'residuals_vvDD_conditions.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Visualize Temporal Dynamics

In [16]:
"""
the C_u, C_i, V dynamics before the injection of virus
for both ctrl and vvDD conditions
"""

import numpy as np

def C_u(t, 
       kappa = scaled_parameters['kappa'],
       rho = scaled_parameters['rho'],
       Cu_0 = 400.):
    # return the value of C_u at time t
    return 1/(kappa + np.exp(-rho*t)*(1/Cu_0 - kappa))

def C_i(t):
    # t has to be between 0 and 1
    # raise error if t is not in the range
    if t.any() < 0 or t.any() > 1:
        raise ValueError("t has to be between 0 and 1")
    # return the value of C_i at time t
    return np.zeros_like(t)

def C_l(t):
    # t has to be between 0 and 1
    if t.any() < 0 or t.any() > 1:
        raise ValueError("t has to be between 0 and 1")
    # return the value of C_l at time t
    return np.zeros_like(t)

def V(t):
    # t has to be between 0 and 1
    if t.any() < 0 or t.any() > 1:
        raise ValueError("t has to be between 0 and 1")
    # return the value of V at time t
    return np.zeros_like(t)

In [23]:
"""
visualize the temporal dynamics of the virus, uninfected and infected tumor cells using the fitted model from the result
from day 3 to day 13
get the simulation results for the optimized parameters
"""

from pypesto.visualize.model_fit import visualize_optimized_model_fit, _get_simulation_rdatas

amici_model = problem.objective.amici_model

species_to_plot = ['C_u', 'C_i', 'C_i1', 'C_i2', 'C_i3', 'C_i4', 'C_i5', 'C_l', 'V']

# simulate from day 3 to day 12
stop_day = 4
timepoints = np.linspace(start=0, stop=stop_day, num=100)

simulation_rdatas = _get_simulation_rdatas(
    result=result,
    problem=problem,
    start_index = 0,
    simulation_timepoints=timepoints,
)


In [ ]:

# Plot all state trajectories
for c_ in range(len(problem.objective.edatas)):
    for species in species_to_plot:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        label = f"{species} - {problem.objective.edatas[c_].id}"
        if problem.objective.edatas[c_].id == 'ctrl':
            ax.plot(timepoints, simulation_rdatas[c_]['x'][:, amici_model.getStateIds().index(species)], color='red', label=label, lw = 3)
        else:
            ax.plot(timepoints, simulation_rdatas[c_]['x'][:, amici_model.getStateIds().index(species)], color='blue', label=label, lw = 3)
        ax.set_ylabel(species)
        ax.set_xlabel('Time (days)')
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Put a legend to the right of the current axis
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.title(f"State {species} over time")
        # Change x-axis range and labels
        ax.set_xticks(np.arange(0, stop_day+1, 1))
        ax.set_xticklabels(np.arange(3, stop_day+4, 1))
        plt.xticks(rotation=60)
        plt.savefig(folder_path + f"{species}_over_time_{problem.objective.edatas[c_].id}.pdf", dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:
"""
visualize the temporal dynamics of the virus, uninfected and infected tumor cells using the fitted model from the result
from day 2 to day 13
"""

# obtain the dynamics of C_u, C_i, V over time
timepoints_previous = np.linspace(start=0, stop=1, num=100)
C_u_previous = C_u(timepoints_previous)
C_i_previous = C_i(timepoints_previous)
C_i1_previous = C_i(timepoints_previous)
C_i2_previous = C_i(timepoints_previous)
C_i3_previous = C_i(timepoints_previous)
C_i4_previous = C_i(timepoints_previous)
C_i5_previous = C_i(timepoints_previous)
C_l_previous = C_l(timepoints_previous)
V_previous = V(timepoints_previous)

C_u_ctrl_whole = C_u(np.concatenate([timepoints_previous, timepoints+1]))

# adjust all state trajectories with previous dynamics
for c_ in range(len(problem.objective.edatas)):
    for species in species_to_plot:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        label = f"{species} - {problem.objective.edatas[c_].id}"
        if problem.objective.edatas[c_].id == 'ctrl':
            ax.plot(timepoints + 1, simulation_rdatas[c_]['x'][:, amici_model.getStateIds().index(species)], color='red', label=label, lw = 3)
            if species == 'C_u':
                ax.plot(timepoints_previous, C_u_previous, color='red', lw = 3)
                ax.plot(np.concatenate([timepoints_previous, timepoints+1]), C_u_ctrl_whole, '--', color='purple', label='C_u - ctrl (analytical)', lw = 3)
            elif species == 'C_i':
                ax.plot(timepoints_previous, C_i_previous, color='red', lw = 3)
            elif species == 'C_i1':
                ax.plot(timepoints_previous, C_i1_previous, color='red', lw = 3)
            elif species == 'C_i2':
                ax.plot(timepoints_previous, C_i2_previous, color='red', lw = 3)
            elif species == 'C_i3':
                ax.plot(timepoints_previous, C_i3_previous, color='red', lw = 3)
            elif species == 'C_i4':
                ax.plot(timepoints_previous, C_i4_previous, color='red', lw = 3)
            elif species == 'C_i5':
                ax.plot(timepoints_previous, C_i5_previous, color='red', lw = 3)
            elif species == 'C_l':
                ax.plot(timepoints_previous, C_l_previous, color='red', lw = 3)
            elif species == 'V':
                ax.plot(timepoints_previous, V_previous, color='red', lw = 3)
        else:
            ax.plot(timepoints + 1, simulation_rdatas[c_]['x'][:, amici_model.getStateIds().index(species)], color='blue', label=label, lw = 3)
            if species == 'C_u':
                ax.plot(timepoints_previous, C_u_previous, color='blue', lw = 3)
                ax.plot(np.concatenate([timepoints_previous, timepoints+1]), C_u_ctrl_whole, '--', color='purple', label='C_u - ctrl (analytical)', lw = 3)
            elif species == 'C_i':
                ax.plot(timepoints_previous, C_i_previous, color='blue', lw = 3)
            elif species == 'C_i1':
                ax.plot(timepoints_previous, C_i1_previous, color='blue', lw = 3)
            elif species == 'C_i2':
                ax.plot(timepoints_previous, C_i2_previous, color='blue', lw = 3)
            elif species == 'C_i3':
                ax.plot(timepoints_previous, C_i3_previous, color='blue', lw = 3)
            elif species == 'C_i4':
                ax.plot(timepoints_previous, C_i4_previous, color='blue', lw = 3)
            elif species == 'C_i5':
                ax.plot(timepoints_previous, C_i5_previous, color='blue', lw = 3)
            elif species == 'C_l':
                ax.plot(timepoints_previous, C_l_previous, color='blue', lw = 3)
            elif species == 'V':
                # ax.set_yscale('log')
                ax.plot(timepoints_previous, V_previous, color='blue', lw = 3)
        
        ax.set_ylabel(species)
        ax.set_xlabel('Time (days)')
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.title(f"State {species} over time")
        # Change x-axis range and labels
        ax.set_xticks(np.arange(0, stop_day+2, 1))
        ax.set_xticklabels(np.arange(2, stop_day+4, 1))
        # rotate the x-axis labels
        plt.xticks(rotation=60)
        
        plt.savefig(folder_path + f"{species}_over_time_{problem.objective.edatas[c_].id}_adjusted.pdf", dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:
# plot the dynamics of the observable tumor volume over time
# for both ctrl and vvDD conditions

for c_ in range(len(problem.objective.edatas)):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(np.concatenate([timepoints_previous, timepoints+1]), result.optimize_result.list[0]['inner_parameters'][0]*C_u_ctrl_whole, '--', color='purple', label='ctrl (analytical)', lw = 3)
    if problem.objective.edatas[c_].id == 'ctrl':
        ax.plot(timepoints+1, simulation_rdatas[c_]['y'].reshape(1,-1)[0], color='red', label='ctrl', lw = 3)
    else:
        ax.plot(timepoints+1, simulation_rdatas[c_]['y'].reshape(1,-1)[0], color='blue', label='vvDD', lw = 3)
    ax.set_ylabel('Tumor Volume ($\mu m^3$)')
    ax.set_xlabel('Time (days)')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(f"Tumor volume over time")
    # Change x-axis range and labels
    ax.set_xticks(np.arange(0, stop_day+2, 1))
    ax.set_xticklabels(np.arange(2, stop_day+4, 1))
    # ax.set_ylim([0, 7e6])
    # change y axis to log scale
    ax.set_yscale('log')
    plt.xticks(rotation=60)
    plt.savefig(folder_path + f"Tumor_volume_over_time_{problem.objective.edatas[c_].id}.pdf", dpi=300, bbox_inches='tight')
    plt.show()

In [37]:
# # access all the species and sum them up to get the total tumor cell number
# # save into a new array
# total_tumor_cells = np.zeros(len(simulation_rdatas[c_]['x'][:,0]))
# for species in ['C_u', 'C_i', 'C_i1', 'C_i2', 'C_i3', 'C_i4', 'C_i5', 'C_l']:
#     total_tumor_cells += simulation_rdatas[0]['x'][:, amici_model.getStateIds().index(species)]

# result.optimize_result.list[0]['inner_parameters'][0]*total_tumor_cells

# Parameter estimation analysis

In [18]:
# get the statistics for 95% CI
cut_off_index = find_cut_off_index(result)
x_trace_within_cut_off = np.array(find_cut_off_x_trace(result, ci = 0.95))
x_trace_within_cut_off_df = pd.DataFrame(x_trace_within_cut_off, columns=problem.x_names)

x_trace_within_cut_off_NF = find_cut_off_x_trace(result, ci = 0.95, flatten=False)
x_trace_within_cut_off_NF_top100 = x_trace_within_cut_off_NF[:100]

# unpack and reshape x_trace_within_cut_off_NF_top100 into an array with the same column number as x_trace_within_cut
x_trace_within_cut_off_NF_top100_unpacked = []
for i in range(len(x_trace_within_cut_off_NF_top100)):
    for j in range(len(x_trace_within_cut_off_NF_top100[i])):
        x_trace_within_cut_off_NF_top100_unpacked.append(x_trace_within_cut_off_NF_top100[i][j])

x_trace_within_cut_off_NF_top100_unpacked_df = pd.DataFrame(x_trace_within_cut_off_NF_top100_unpacked, columns=problem.x_names)

In [ ]:
plt.rcParams.update({'font.size': 30})

waterfall(result, size=(wid, hei))
plt.savefig(os.path.join(folder_path, 'waterfall_plot.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

In [ ]:
pypesto.visualize.optimizer_history(result)
plt.savefig(os.path.join(folder_path, 'optimizer_history_plot.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 30})

fig, axs = plt.subplots(1, 3, figsize=(wid, hei), sharey=False, )

pypesto.visualize.parameters(result, ax = axs[0], plot_inner_parameters=False, start_indices=cut_off_index,  colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', cut_off_index))
pypesto.visualize.parameters(result, ax = axs[1], plot_inner_parameters=False, start_indices=300,  colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 300))
pypesto.visualize.parameters(result, ax = axs[2], plot_inner_parameters=False, start_indices=100,  colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 100))

axs[0].set_title('95% CI', fontsize=30)
axs[1].set_title('Top 300', fontsize=30)
axs[2].set_title('Top 100', fontsize=30)

# set all the x axis, x and y labels to have fontsize 30
for j in range(3):
    axs[j].set_xticklabels(axs[j].get_xticklabels(), fontsize=20)
    axs[j].set_xlabel('Parameter Value', fontsize=30)
    axs[j].set_ylabel('Parameter', fontsize=30)

plt.suptitle('Parameter Estimation', fontsize=30)
plt.tight_layout()
plt.savefig(os.path.join(folder_path, 'parameters_plot.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

In [ ]:
pypesto.visualize.optimization_scatter(result, start_indices=cut_off_index, size=(wid, hei))
plt.savefig(os.path.join(folder_path, 'parameters_scatter_plot_cut_off_'+str(cut_off_index)+'.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

# For publication adjustment

In [ ]:
# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# Panel a
for i in range(10):
    axs[0].plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='red', alpha=0.3, label='ctrl (data)' if i == 0 else "")
    axs[0].plot(timepoints+3, simulation_rdatas[i]['y'], 
         linestyle='-', lw= 3, color='red', alpha=1, label='ctrl (model)' if i == 0 else "")

for i in range(10, 20):
    axs[0].plot(np.array([3,4,5,6,7]), data[i], 
             marker='o', linestyle='-', color='blue', alpha=0.3, label='vvDD (data)' if i == 10 else "")
    axs[0].plot(timepoints+3, simulation_rdatas[i]['y'], 
         linestyle='-', lw= 3, color='blue', alpha=1, label='vvDD (model)' if i == 10 else "")

axs[0].set_xlabel('Time (days)')
axs[0].set_ylabel(r'Tumor Volume ($\mu m^3$)')
axs[0].set_title('Model vs data (individual)', pad=20, loc='center')
axs[0].grid(False)
axs[0].set_xticks([3, 4, 5, 6, 7])
axs[0].text(-0.2, 1.25, 'a', transform=axs[0].transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

# Remove top and right lines
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)

# Make spines thicker
axs[0].spines['left'].set_linewidth(2)
axs[0].spines['bottom'].set_linewidth(2)

# Make ticks thicker
axs[0].tick_params(width=2)

# Panel b
pbs_mean = np.array(data[0:10]).mean(axis=0)
pbs_se = np.array(data[0:10]).std(axis=0)
# Calculate mean and standard error for vvDD
vvDD_mean = np.array(data[10:20]).mean(axis=0)
vvDD_se = np.array(data[10:20]).std(axis=0)

# Plot provided data
axs[1].errorbar(np.array([3,4,5,6,7]), pbs_mean, 
             yerr=[pbs_mean - pbs_se],
             fmt='o--', color='red', alpha=0.3, ecolor='red', capsize=8, label='ctrl (data)')
axs[1].errorbar(np.array([3,4,5,6,7]), vvDD_mean, 
             yerr=[vvDD_mean - vvDD_se],
             fmt='o--', color='blue', alpha=0.3, ecolor='blue', capsize=8, label='vvDD (data)')

# put simulattion rdatas into an array
simulation_rdatas_array = []
for i in range(20):
    simulation_rdatas_array.append(simulation_rdatas[i]['y'].reshape(1,-1)[0])

# Calculate mean and standard error for PBS
pbs_mean_sim = np.array(simulation_rdatas_array[0:10]).mean(axis=0)
pbs_se_sim = np.array(simulation_rdatas_array[0:10]).std(axis=0)
# Calculate mean and standard error for vvDD
vvDD_mean_sim = np.array(simulation_rdatas_array[10:20]).mean(axis=0)
vvDD_se_sim = np.array(simulation_rdatas_array[10:20]).std(axis=0)
# Plot simulated data
# Plot simulated data with shaded area for standard error
axs[1].plot(timepoints+3, pbs_mean_sim, 
             linestyle='-', lw=3,color='red', alpha=1, label='ctrl (model)')
axs[1].fill_between(timepoints+3, pbs_mean_sim - pbs_se_sim, pbs_mean_sim + pbs_se_sim, 
             color='red', alpha=0.1)

axs[1].plot(timepoints+3, vvDD_mean_sim, 
             linestyle='-', lw=3, color='blue', alpha=1, label='vvDD (model)')
axs[1].fill_between(timepoints+3, vvDD_mean_sim - vvDD_se_sim, vvDD_mean_sim + vvDD_se_sim, 
             color='blue', alpha=0.1)

axs[1].set_xlabel('Time (days)')
axs[1].set_ylabel(r'Tumor Volume ($\mu m^3$)') 
axs[1].set_title('Model vs data (mean and std)', pad=20, loc='center')
axs[1].grid(False)
axs[1].set_xticks([3, 4, 5, 6, 7])
axs[1].text(-0.2, 1.25, 'b', transform=axs[1].transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

# Remove top and right lines
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)

# Make spines thicker
axs[1].spines['left'].set_linewidth(2)
axs[1].spines['bottom'].set_linewidth(2)

# Make ticks thicker
axs[1].tick_params(width=2)

# Adjust layout and save the figure
# Create a legend for each subplot
handles, labels = axs[0].get_legend_handles_labels()
handles2, labels2 = axs[1].get_legend_handles_labels()
all_handles = handles + handles2
all_labels = labels + labels2

# Separate handles and labels for red and blue
red_handles = [h for h, l in zip(all_handles, all_labels) if 'ctrl' in l]
red_labels = [l for l in all_labels if 'ctrl' in l]
blue_handles = [h for h, l in zip(all_handles, all_labels) if 'vvDD' in l]
blue_labels = [l for l in all_labels if 'vvDD' in l]

# Create legends
fig.legend(red_handles, red_labels, loc='lower center', ncol=2, frameon=False, bbox_to_anchor=(0.25, -0.2))
fig.legend(blue_handles, blue_labels, loc='lower center', ncol=2, frameon=False, bbox_to_anchor=(0.75, -0.2))
plt.tight_layout()
plt.subplots_adjust(wspace=0.6)  # Add more space between subplots
plt.savefig(folder_path + 'combined_fit.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Create gradient color maps
red_cmap = LinearSegmentedColormap.from_list("red_gradient", ["#ffcccc", "#ff0000"])
blue_cmap = LinearSegmentedColormap.from_list("blue_gradient", ["#ccccff", "#0000ff"])

# Create a 3x2 subplot layout with shared y-axis for each row
fig, axs = plt.subplots(9, 2, figsize=(wid, hei), sharex=True, sharey='row')

panel_labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

for i, species in enumerate(species_to_plot):
    for c_, condition in enumerate(['ctrl', 'vvDD']):
        ax = axs[i, c_]
        color_map = red_cmap if condition == 'ctrl' else blue_cmap
        for j in range(10):
            color = color_map(j / 9)  # Normalize j to the range [0, 1]
            ax.plot(timepoints, simulation_rdatas[c_ * 10 + j]['x'][:, amici_model.getStateIds().index(species)], color=color, lw=2)
        
        ax.set_xticks(np.arange(0, stop_day + 1, 1))
        ax.set_xticklabels(np.arange(3, stop_day + 4, 1))
        if i == 0:
            ax.set_title(f"{condition}", pad=20, loc='center')
            ax.set_ylabel(r'$C_u$', labelpad=20)
        else:
            ax.set_title(f" ", pad=-10, loc='center')
        if i == 1:
            ax.set_ylabel(r'$C_{i,0}$', labelpad=20)
        if i == 2:
            ax.set_ylabel(r'$C_{i,1}$', labelpad=20)
        if i == 3:
            ax.set_ylabel(r'$C_{i,2}$', labelpad=20)
        if i == 4:
            ax.set_ylabel(r'$C_{i,3}$', labelpad=20)
        if i == 5:
            ax.set_ylabel(r'$C_{i,4}$', labelpad=20)
        if i == 6:
            ax.set_ylabel(r'$C_{i,5}$', labelpad=20)
        if i == 7:
            ax.set_ylabel(r'$C_l$', labelpad=20)
        if i == 8:
            ax.set_ylabel(r'$V$', labelpad=20)
            ax.set_xlabel('Time (days)')

        ax.yaxis.set_tick_params(labelleft=True)
        
        # Remove top and right lines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # Make spines thicker
        ax.spines['left'].set_linewidth(2)
        ax.spines['bottom'].set_linewidth(2)

        # Make ticks thicker
        ax.tick_params(width=2)
        if i == 0:
            # Add panel labels
            ax.text(-0.15, 2, panel_labels[i * 2 + c_], transform=ax.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

# Adjust the position of the y-axis labels
for ax in axs[:, 0]:
    ax.yaxis.set_label_coords(-0.125, 0.4)
# Adjust the position of the y-axis labels
for ax in axs[:, 1]:
    ax.yaxis.set_label_coords(-0.125, 0.4)

plt.subplots_adjust(wspace=0.3, hspace=0.8)  # Adjust space between subplots
plt.savefig(folder_path + 'state_trajectories.pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
plt.rcParams.update({'font.size': 30})

fig = plt.figure(figsize=(wid, hei+10))

gs = fig.add_gridspec(2, 3)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 2])

waterfall(result, ax=ax1)
# ax1.set_ylabel('Objective value\n(offset=-1.545e+03)')

# Parameter plots in the second row
pypesto.visualize.parameters(result, ax=ax2, plot_inner_parameters=False, start_indices=cut_off_index, colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', cut_off_index))
pypesto.visualize.parameters(result, ax=ax3, plot_inner_parameters=False, start_indices=300, colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 300))
pypesto.visualize.parameters(result, ax=ax4, plot_inner_parameters=False, start_indices=100, colors=hex_to_rgba_gradient('#A7C9F8', '#28518B', 100))

ax2.set_title('95% CI', fontsize=30)
ax3.set_title('Top 300', fontsize=30)
ax4.set_title('Top 100', fontsize=30)

# Set all the x-axis, x and y labels to have fontsize 30
for ax in [ax2, ax3, ax4]:
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=20)
    ax.set_xlabel('Parameter Value', fontsize=30)
ax2.set_ylabel('Parameter', fontsize=30)

# Remove top and right lines and make lines and ticks thicker
for i, ax in enumerate([ax1, ax2, ax3, ax4]):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.tick_params(width=2)
    # Add panel labels
    if i == 0:
        ax.text(-0.115, 1, panel_labels[i], transform=ax.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')
    else:
        ax.text(-0.55, 1, panel_labels[i], transform=ax.transAxes, fontsize=40, fontweight='bold', va='top', ha='right')

plt.tight_layout()
plt.savefig(os.path.join(folder_path, 'waterfall_parameters_plot.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

# Profiling

In [ ]:
result = profile.parameter_profile(
    problem=problem,
    result=result,
    engine=MultiProcessEngine(),
    optimizer=ScipyOptimizer(),
    profile_options=ProfileOptions(whole_path=True),
    overwrite=True,
    filename='optimization_history/n5000.hdf5',
)

In [ ]:
profiles(result, show_bounds=True, size=(wid, hei))
plt.savefig(os.path.join(folder_path, 'profile_whole_path_plot.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

# Sampling

In [ ]:
# sampling
sampler = pypesto.sample.AdaptiveParallelTemperingSampler(
    internal_sampler=pypesto.sample.AdaptiveMetropolisSampler(), n_chains=5
    )
sampling_n = 10000
result = pypesto.sample.sample(
    problem,
    n_samples=sampling_n,
    sampler=sampler,
    result=result,
    filename='optimization_history/n5000.hdf5',
)

In [ ]:
ax = pypesto.visualize.sampling_parameter_traces(
    result,
    use_problem_bounds=False,
    size=(wid, hei),
)
plt.savefig(os.path.join(folder_path, 'params_traceplot_sampling'+str(sampling_n)+'.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

pypesto.visualize.sampling_1d_marginals(result, size=(wid, hei))
plt.savefig(os.path.join(folder_path, 'params_marginalplot_sampling'+str(sampling_n)+'.pdf'), dpi=dpi, bbox_inches="tight")
plt.show()

In [ ]:
# the problem is that the release of virus is too fast, and the peak value is
# QUESTION: but how to contraint it properly? based on just the data, all the simulation is within the error bar

# TODO: update overleaf documentation
# TODO: organize the notes in all papers
# TODO: Notion goals organzing

# TODO: mixed effects model exploration